In [1]:
import arxiv
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List
from langchain.tools import StructuredTool

In [2]:
class AnswerQuestionInput(BaseModel):
    question: str = Field(description="The question to answer")
    abstracts: List[str] = Field(description="The abstracts to use")

# Create some tools that can be used by the agent
def get_absracts(query: str) -> list[str]:
    search = arxiv.Search(query, max_results=4)
    return [result.summary for result in search.results()]


def answer_question(abstracts: list[str], question: str) -> str:
    template = (
        "You are a researcher at Oxford University. Your goal is to answer "
        "a question. You can use the following abstracts to help you: "
        "Abstracts: {abstracts}. Question: {question}. Answer: "
    )

    prompt_template = PromptTemplate(
        template=template,
        input_variables=["abstracts", "question"],
    )

    llm_chain = LLMChain(
        llm=OpenAI(temperature=0),
        prompt=prompt_template,
    )

    response = llm_chain.predict(
        abstracts="\n\n".join(abstracts),
        question=question,
    )

    return response

In [3]:
# Define the list of available tools
tools = [
    Tool(
        name="arXiv Search",
        func=get_absracts,
        description="useful for finding research abstracts given a query",
    ),
    StructuredTool.from_function(
        name="Answer Question from Abstracts",
        func=answer_question,
        description="useful for answering a question given some abstracts",
        args_schema=AnswerQuestionInput,
    ),
]

# Instantiate the agent
memory = ConversationBufferMemory()
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

zero_shot_agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
)

response = zero_shot_agent.run(
    "Get some abstracts about quantum computing. Then answer the question: What is quantum computing?"
)



> Entering new AgentExecutor chain...
Thought: To find abstracts about quantum computing, I can use the arXiv Search tool. Once I have the abstracts, I can use the Answer Question from Abstracts tool to answer the question "What is quantum computing?"

Action:
```
{
  "action": "arXiv Search",
  "action_input": {
    "query": "quantum computing"
  }
}
```
Observation: ['This article highlights quantum Internet computing as referring to\ndistributed quantum computing over the quantum Internet, analogous to\n(classical) Internet computing involving (classical) distributed computing over\nthe (classical) Internet. Relevant to quantum Internet computing would be areas\nof study such as quantum protocols for distributed nodes using quantum\ninformation for computations, quantum cloud computing, delegated verifiable\nblind or private computing, non-local gates, and distributed quantum\napplications, over Internet-scale distances.', "This paper investigates a variety of unconventional quant

In [4]:
response

'Quantum computing is a form of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It is based on the principles of quantum mechanics, which describe the behavior of matter and energy at the atomic and subatomic level. Quantum computing is used to solve complex problems that are intractable for classical computers, such as simulating quantum systems, searching large databases, and factoring large numbers. It has the potential to revolutionize computing, with applications in areas such as cryptography, artificial intelligence, and materials science.'